# KNN

In [78]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Cargar los datos de train y test
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')


In [79]:
# Crear una matriz de usuario-item para los datos de entrenamiento
train_matrix = train.pivot(index='user', columns='item', values='rating').fillna(0)

C:\Users\User\AppData\Local\Temp\ipykernel_15444\729011336.py:2: PerformanceWarning: The following operation may generate 12573536976 cells in the resulting pandas object.
  train_matrix = train.pivot(index='user', columns='item', values='rating').fillna(0)


MemoryError: Unable to allocate 11.7 GiB for an array with shape (12573536976,) and data type bool

In [81]:
train

,user,item,rating
0,1,25715,7.0
1,1,25716,10.0
2,5,25851,9.0
3,6,25923,5.0
4,7,25924,6.0
...,...,...,...
390346,77801,185970,5.0
390347,77801,185971,5.0
390348,77802,44671,10.0
390349,77803,154450,10.0


In [82]:
test

,ID,user,item
0,0,8117,268
1,1,10512,24393
2,2,534,1334
3,3,10984,6550
4,4,9093,22128
...,...,...,...
43315,43315,534,1751
43316,43316,1150,5467
43317,43317,10184,8805
43318,43318,7531,11566


In [86]:
train = train.sample(n=1000, random_state=42)

# Filtrar el conjunto de test para que contenga solo los usuarios e ítems presentes en el conjunto de entrenamiento
users_in_train = train['user'].unique()
items_in_train = train['item'].unique()

test = test[test['user'].isin(users_in_train) & test['item'].isin(items_in_train)].sample(n=100, random_state=42)

In [87]:
mean_rating = train['rating'].mean()

# Crear una matriz de usuario-item para los datos de entrenamiento
train_matrix = train.pivot(index='user', columns='item', values='rating').fillna(mean_rating)

In [88]:
train_matrix

item,44,47,58,61,76,97,119,138,165,169,...,182202,182642,182660,182712,183575,184315,184553,184587,184874,185950
user,,,,,,,,,,,,,,,,,,,,,
143,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,...,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635
203,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,...,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635
360,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,...,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635
534,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,...,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635
746,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,...,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77260,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,...,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635
77269,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,...,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635
77417,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,...,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635,7.635


In [89]:
# Función para calcular la similitud de Jaccard entre dos usuarios
def jaccard_similarity(user1, user2, train_matrix):
    # Obtener los ítems calificados por cada usuario
    items_user1 = set(train_matrix.loc[user1][train_matrix.loc[user1] > 0].index)
    items_user2 = set(train_matrix.loc[user2][train_matrix.loc[user2] > 0].index)
    
    # Calcular la intersección y la unión
    intersection = len(items_user1.intersection(items_user2))
    union = len(items_user1.union(items_user2))
    
    # Retornar la similaridad de Jaccard
    return intersection / union if union != 0 else 0

In [90]:
# Crear la matriz de similitud de Jaccard para todos los pares de usuarios
def create_similarity_matrix(train_matrix):
    users = train_matrix.index
    sim_matrix = np.zeros((len(users), len(users)))

    for i, user1 in enumerate(users):
        for j, user2 in enumerate(users):
            if i <= j:  # Solo calcular la mitad superior de la matriz (simétrica)
                sim_matrix[i, j] = sim_matrix[j, i] = jaccard_similarity(user1, user2, train_matrix)
    
    return sim_matrix, users

In [91]:
def predict_ratings(test, sim_matrix, train_matrix, k=25):
    predictions = []

    for _, row in test.iterrows():
        user = row['user']
        item = row['item']
        
        # Encontrar los k usuarios más similares
        user_index = train_matrix.index.get_loc(user)
        similarities = sim_matrix[user_index]
        
        # Obtener los índices de los k usuarios más similares
        similar_users = np.argsort(similarities)[-k:]
        
        # Calcular la predicción como un promedio ponderado de las calificaciones de los usuarios similares
        weighted_ratings = 0
        total_weight = 0
        for sim_user_index in similar_users:
            sim_user = train_matrix.index[sim_user_index]
            if train_matrix.loc[sim_user, item] > 0:  # Solo considerar calificaciones existentes
                weighted_ratings += similarities[sim_user_index] * train_matrix.loc[sim_user, item]
                total_weight += similarities[sim_user_index]
        
        # Si no hay usuarios similares con calificaciones, predecir la media global
        if total_weight > 0:
            prediction = weighted_ratings / total_weight
        else:
            prediction = train_matrix[item].mean()  # Usamos la media de la columna (por ítem)
        
        predictions.append(prediction)
    
    return predictions

In [92]:
# Crear la matriz de similaridad de Jaccard
sim_matrix, users = create_similarity_matrix(train_matrix)

In [ ]:
# Realizar las predicciones para el conjunto de prueba
predictions = predict_ratings(test, sim_matrix, train_matrix, k=25)
predictions_normalized = np.clip(predictions, 1, 10)

# Guardar las predicciones en un archivo CSV
test['rating'] = predictions_normalized

test[['ID', 'rating']].to_csv('submision-KNN-Small.csv', index=False)

print("Predicciones guardadas en 'submision-KNN-Small.csv'")

Predicciones guardadas en 'submision.csv'


# CONCLUSION

El uso de algoritmos de recomendación basados en similitud, como KNN , presenta desafíos importantes cuando se manejan grandes volúmenes de datos. Las matrices de usuario-item pueden ser extremadamente grandes, lo que consume mucha memoria y puede llevar a errores de carga. Al reducir el tamaño de los datos para evitar estos problemas, se corre el riesgo de perder información clave, lo que afecta negativamente la precisión de las recomendaciones. Para abordar esto, es recomendable usar técnicas como matrices dispersas, reducción de dimensionalidad o modelos de factorización, que permiten manejar grandes cantidades de datos de manera eficiente sin comprometer demasiado la calidad de las recomendaciones.
No obstatnte en este caso para la competición este enfoque de sistema de recomendación se considera que no es el óptimo pero nos ha ayudado a ver las limitaciones y contradicciones de este modelo.